In [8]:
import numpy as np 
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE

In [9]:
df = pd.read_csv ("diabetes.csv")

In [10]:
df.shape

(78094, 126)

In [11]:
Y = df['diabetes_mellitus'] ## Label, target variable, 0 --> doesn't have diabetes_mellitus, 
                                                        #   1 --> does have diabetes_mellitus
X = df.drop(columns=['diabetes_mellitus']) ## 125 feature data set

In [12]:
X.shape

(78094, 125)

In [13]:
Y.shape

(78094,)

In [ ]:
def preprocess_data(train_data, test_data):
    remove_columns = ["hospital_id", "nan_counts", "icu_id"]
    train_data.drop(remove_columns, axis=1, inplace=True)
    test_data.drop(remove_columns, axis=1, inplace=True)

    y_train = train_data['diabetes_mellitus']
    x_train = train_data.drop(columns=['diabetes_mellitus'])

    numeric_cols = x_train.select_dtypes(include=['int64', 'float64']).columns
    categorical_cols = x_train.select_dtypes(include=['object']).columns

    imputer = SimpleImputer(strategy='mean')
    x_train[numeric_cols] = imputer.fit_transform(x_train[numeric_cols])
    test_data[numeric_cols] = imputer.transform(test_data[numeric_cols])

    scaler = StandardScaler()
    x_train[numeric_cols] = scaler.fit_transform(x_train[numeric_cols])
    test_data[numeric_cols] = scaler.transform(test_data[numeric_cols])

    x_train_encoded = pd.get_dummies(x_train, columns=categorical_cols, drop_first=True)
    test_data_encoded = pd.get_dummies(test_data, columns=categorical_cols, drop_first=True)

    x_train_encoded, test_data_encoded = x_train_encoded.align(test_data_encoded, join='inner', axis=1, fill_value=0)

    return x_train_encoded, y_train, test_data_encoded

In [ ]:
def save_submission(predictions_binary, test_data, filename='Sample_SubmissionSVM.csv'):
    submission_df = pd.DataFrame()
    submission_df['ID'] = test_data['encounter_id']
    submission_df['diabetes_mellitus'] = predictions_binary

    submission_df.to_csv(filename, index=False)

In [14]:

train_data = pd.read_csv("diabetes.csv")
test_data = pd.read_csv("diabetes_test_unlabeled.csv")
test_encounter_id = pd.read_csv("diabetes_test_unlabeled.csv")

x_train_encoded, y_train, test_data_encoded = preprocess_data(train_data, test_data)
x_train, x_val, y_train, y_val = train_test_split(x_train_encoded, y_train, test_size=0.3, random_state=42)

clf = SVC()

clf.fit(x_train, y_train)
y_pred = clf.predict(x_val)
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy score: ", accuracy)
test_predictions_binary = clf.predict(test_data_encoded)
save_submission(test_predictions_binary, test_encounter_id, filename='Diabetes_mellitus Using SVM_Predictions.csv')


Accuracy score:  0.8172777327244014
